In [1]:
if (!require("rjson")) {
  install.packages("rjson")
  stopifnot(require("rjson"))
}
if (!require("jsonlite")) {
  install.packages("jsonlite")
  stopifnot(require("jsonlite"))
}
if (!require("stringr")) {
   install.packages("stringr")
   stopifnot(require("stringr"))
}
if (!require("sqldf")) {
   install.packages("sqldf")
   stopifnot(require("sqldf"))
}
if (!require("kernlab")) {
   install.packages("kernlab")
   stopifnot(require("kernlab"))
}
if (!require("tidytext")) {
   install.packages("tidytext")
   stopifnot(require("tidytext"))
}
if (!require("dplyr")) {
   install.packages("dplyr")
   stopifnot(require("dplyr"))
}
if (!require("TSA")) {
   install.packages("TSA")
   stopifnot(require("TSA"))
}
if (!require("tidyverse")) {
   install.packages("tidyverse")
   stopifnot(require("tidyverse"))
}
if (!require("plotly")) {
   install.packages("plotly")
   stopifnot(require("plotly"))
}
if (!require("spacyr")) {
   install.packages("spacyr")
   stopifnot(require("spacyr"))
}

Loading required package: rjson
Loading required package: jsonlite

Attaching package: ‘jsonlite’

The following objects are masked from ‘package:rjson’:

    fromJSON, toJSON

Loading required package: stringr
Loading required package: sqldf
Loading required package: gsubfn
Loading required package: proto
Warning message in system2("/usr/bin/otool", c("-L", shQuote(DSO)), stdout = TRUE):
“running command ''/usr/bin/otool' -L '/Users/milicacvetkovic/anaconda3/envs/r-tutorial/lib/R/library/tcltk/libs//tcltk.so'' had status 1”Loading required package: RSQLite
Loading required package: kernlab
Loading required package: tidytext
Loading required package: dplyr

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: TSA

Attaching package: ‘TSA’

The following objects are masked from ‘package:stats’:

    acf, arima

The foll

## 1. Read the data

In [2]:
business<-jsonlite::stream_in(file("data/business_city.json"))
review<-jsonlite::stream_in(file("data/review_city.json"))
tip<-jsonlite::stream_in(file("data/tip_city.json"))
user<-jsonlite::stream_in(file("data/user_city.json"))

opening file input connection.


 Imported 36327 records. Simplifying...


closing file input connection.
opening file input connection.


 Imported 942027 records. Simplifying...


closing file input connection.
opening file input connection.


 Imported 129571 records. Simplifying...


closing file input connection.
opening file input connection.


 Imported 272014 records. Simplifying...


closing file input connection.


## 2. Create the accessors

In [3]:
#First, we need to find all the pubs by using the tag:alcohol in the business
#Then we need to decide what tags to stay.
all_pubs<-data.frame()
for (i in length(business$name):1)
  {
   
   judgement_pre<-strsplit(business$attributes$Alcohol[i],"'")[[1]]
   judgement<-judgement_pre[2]
   if(is.na(business$attributes$Alcohol[i])==0 &is.na(judgement)==0)
   {
   if(judgement=="full_bar" | business$attributes$Alcohol[i]=="full_bar" )
     {
     #find the pub and record it.
    all_pubs_pre<-data.frame(name=business$name[i],business_id=business$business_id[i],review_count=business$review_count[i],is.open=business$is_open[i],stars=business$stars[i],attributes=business$attributes[i,],hours=business$hours[i,],categories=business$categories[i],state=business$state[i],city=business$city[i])
    all_pubs<-rbind(all_pubs,all_pubs_pre)
   }
   }
}

In [4]:
#Now we have all the data of pubs.
#Next we need to find all open pubs

all_pubs<-subset(all_pubs, is.open==1)

In [6]:
#Then we will find the reviews that are related to the pubs.(It will be a long process to run the code below)

all_review<-subset(review,business_id%in%all_pubs$business_id)

In [7]:
head(all_review)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
1,LG2ZaYiOgpr2DK_90pYjNw,V34qejxNsCbcgD8C0HVk-Q,HQl28KMwrEKHqhFrrDqVNQ,5,1,0,0,"I love Deagan's. I do. I really do. The atmosphere is cozy and festive. The shrimp tacos and house fries are my standbys. The fries are sometimes good and sometimes great, and the spicy dipping sauce they come with is to die for. The beer list is amazing and the cocktails are great. The prices are mid-level, so it's not a cheap dive you can go to every week, but rather a treat when you do. Try it out. You won't be disappointed!",2015-12-05 03:18:11
4,2mLlQ9n5Ln_fykOuX_b0-A,EZ6g9a17UdMjtR-wrZebmA,wSAGILK2CFbq5BVZ5D67-A,3,0,0,0,The service is attentive. The decor attractive. But the menu is far too doused in polenta and paprika. The fingerling potatoes? Drenched in paprika. The aioli?Mixed with paprika. The appetizer? Fried polenta. The fish? Polenta crusted. It's worth a try. But be forewarned. The chef must shop at Costco.,2015-06-21 00:07:59
7,LzCJ5m9xbiSJElYfwufqqQ,rHC67vLw-_pwoGjmEn-iQA,u8CFMZ9rCMDvKGDyqt6MEQ,5,0,0,0,If you are looking for a fresh yummy breakfast on the weekend in Waunakee you must try Cuco's. They have your traditional American breakfast choices or some tasty Mexican dishes. The food is fresh and comes out sizzling hot. The service is excellent and the restaurant is very clean.,2017-05-13 15:10:59
17,h3H1A-fzNNC8ppKeNwkkhg,q3ycagiLqUaRkOgEPf_gbw,ODTI9W7fHMKNWZ3g8VcFUg,4,1,0,1,"What a great restaurant in the South Side! We went for restaurant week but the entrees are on their normal menu. Everything had such a pop of flavor and was perfectly cooked. The only part of our meal that my boyfriend didn't enjoy was the dessert. The shrimp appetizer is loaded with garlic! Definitely enough for two people. The stuffed oysters were very filling but I liked the shrimp more. For entrees I had the chicken in cream sauce which was amazing. My boyfriend had a perfectly cooked fillet. My lemon cake was awesome but his creme brulee was watery. They substituted it for him. Awesome service, great wine, and great food! We will be back!",2017-01-14 23:23:29
18,bviGAY3LUNOcOFaI6f_HDA,1EADjPQ3-qu7DSQ_8mII2Q,hsDkxhc-_YYsVqAeQBJJjg,5,2,0,1,"I saw Big Jim's on Diners, Drive-ins and Dives. Went there when in Pittsburgh with my son visiting Pitt, had a great lunch. My son went to Pitt and I have been back several times. The food is very good, large servings and reasonably priced. This place will surprise you. It is off the beaten path. You have to be looking for it because it is below the highway in a neighborhood that leads nowhere to my knowledge. It is unattractive at best but the service is good, food is very good, fresh, plentiful and very reasonable. If you like good food, funky places and you are not a snob stop in and give it a try.",2012-02-01 06:25:35
23,lFGpBaCGBWzjlTiS5_I3PQ,iWsZrhQNLDivSrypzpfdBg,APPZiUz_CEvR_--qY-tbig,4,0,0,0,Great place! This is our go to place for sushi in CLE. They have a tuna appetizer that is served in the shape of a heart and is possibly the most refreshing and delicious thing on a hot summer night!,2014-06-11 15:50:27


In [8]:
#Here we got all the related reviews.
#Then we will do the same thing to the tip and user

all_tip<-subset(tip,business_id%in%all_pubs$business_id)

user_id_pre<-unique(all_review$user_id)

all_user<-subset(user,user_id%in%user_id_pre)

In [9]:
#In case that there are same comments, we decide to delete all the repeated data.

sq1 <- sqldf("select text, count(text ) count from all_review group by text having count > 1")
all_review<-subset(all_review,!(text%in%sq1$text))
all_user<-subset(all_user,user_id%in%all_review$user_id)
all_tip<-subset(all_tip,user_id%in%all_user$user_id)

In [10]:
colnames(all_pubs)
colnames(all_tip)
colnames(all_review)
colnames(all_user)

[1] "name"                                 
 [2] "business_id"                          
 [3] "review_count"                         
 [4] "is.open"                              
 [5] "stars"                                
 [6] "attributes.RestaurantsAttire"         
 [7] "attributes.RestaurantsTakeOut"        
 [8] "attributes.BusinessAcceptsCreditCards"
 [9] "attributes.NoiseLevel"                
[10] "attributes.GoodForKids"               
[11] "attributes.RestaurantsReservations"   
[12] "attributes.RestaurantsGoodForGroups"  
[13] "attributes.BusinessParking"           
[14] "attributes.RestaurantsPriceRange2"    
[15] "attributes.HasTV"                     
[16] "attributes.Alcohol"                   
[17] "attributes.BikeParking"               
[18] "attributes.RestaurantsDelivery"       
[19] "attributes.ByAppointmentOnly"         
[20] "attributes.OutdoorSeating"            
[21] "attributes.Caters"                    
[22] "attributes.WheelchairAccessible"      
[23] "attributes.WiFi"                      
[24] "attributes.Ambience"                  
[25] "attributes.BYOB"                      
[26] "attributes.GoodForMeal"               
[27] "attributes.Corkage"                   
[28] "attributes.RestaurantsTableService"   
[29] "attributes.BusinessAcceptsBitcoin"    
[30] "attributes.HappyHour"                 
[31] "attributes.DogsAllowed"               
[32] "attributes.CoatCheck"                 
[33] "attributes.Music"                     
[34] "attributes.BestNights"                
[35] "attributes.HairSpecializesIn"         
[36] "attributes.AcceptsInsurance"          
[37] "attributes.BYOBCorkage"               
[38] "attributes.DriveThru"                 
[39] "attributes.GoodForDancing"            
[40] "attributes.Smoking"                   
[41] "attributes.AgesAllowed"               
[42] "attributes.DietaryRestrictions"       
[43] "attributes.RestaurantsCounterService" 
[44] "attributes.Open24Hours"               
[45] "hours.Monday"                         
[46] "hours.Tuesday"                        
[47] "hours.Wednesday"                      
[48] "hours.Thursday"                       
[49] "hours.Friday"                         
[50] "hours.Saturday"                       
[51] "hours.Sunday"                         
[52] "categories"                           
[53] "state"                                
[54] "city"

[1] "user_id"          "business_id"      "text"             "date"            
[5] "compliment_count"

[1] "review_id"   "user_id"     "business_id" "stars"       "useful"     
[6] "funny"       "cool"        "text"        "date"

[1] "user_id"            "name"               "review_count"      
 [4] "yelping_since"      "useful"             "funny"             
 [7] "cool"               "elite"              "friends"           
[10] "fans"               "average_stars"      "compliment_hot"    
[13] "compliment_more"    "compliment_profile" "compliment_cute"   
[16] "compliment_list"    "compliment_note"    "compliment_plain"  
[19] "compliment_cool"    "compliment_funny"   "compliment_writer" 
[22] "compliment_photos"

In [11]:
dim(all_tip)
dim(all_review)
dim(all_pubs)

[1] 35943     5

[1] 314845      9

[1] 3078   54

Remove the columns not related to the restaurants

In [12]:
drops <- c("attributes.HairSpecializesIn","attributes.AcceptsInsurance")
all_pubs = all_pubs[ , !(names(all_pubs) %in% drops)]

In [13]:
head(all_pubs['categories'])

,categories
36324,"Shopping, Party & Event Planning, Caterers, Toy Stores, Pizza, Event Planning & Services, Chicken Wings, Restaurants"
36319,"Restaurants, Barbeque, Comfort Food, Salad, Event Planning & Services, Caterers, Burgers, American (Traditional), Sandwiches"
36303,"Restaurants, Mexican"
36291,"Restaurants, Seafood, Breakfast & Brunch, American (Traditional)"
36275,"Restaurants, Coffee & Tea, Fast Food, Burgers, Food"
36258,"Restaurants, Greek"


In [14]:
table(all_pubs$city)


             Madison        North Olmsted          Rocky River 
                 311                   25                   20 
          Pittsburgh               Medina                Akron 
                 650                   34                   61 
           Homestead            Twinsburg            Champaign 
                  17                   10                   78 
              Burton           Willoughby               Mentor 
                   5                   31                   44 
            Downtown          Monroeville             Lakewood 
                   1                   33                   63 
   Broadview Heights                 Kent            Mcfarland 
                   9                   29                    0 
          Brook Park            Cleveland            Brookpark 
                   5                  396                    1 
        Strongsville            Fitchburg            Wadsworth 
                  30                   

In [15]:
table(all_pubs$state)


  WI   OH   PA   IL 
 466 1473 1020  119 

In [16]:
table(all_pubs$categories)


                                                                                                                                                                                                Nightlife, Diners, Bars, Southern, Restaurants 
                                                                                                                                                                                                                                             0 
                                                                                                                          Shopping, Party & Event Planning, Caterers, Toy Stores, Pizza, Event Planning & Services, Chicken Wings, Restaurants 
                                                                                                                                                                                                                                             1 
                                       

In [17]:
head(all_pubs$review_count)

[1]  19 120  84 451  13  18

Create a new dataset that is going to have 'name' 'business_id' 'review_count' 'is.open' 'stars' and each of of the categories columns. The categories are going to have 1 or 0 values. We will do the same for the location. This way when we build an app, a customer can filter by location and by category.
Then extract this to a new dataset pubs_categories

In [19]:
pubs_categories = all_pubs %>%
    select(name, business_id, review_count, stars, categories, state) %>% 
    filter(review_count>3) %>%
    group_by(business_id) %>%
    unnest_tokens(categories, categories, token = 'regex', pattern=", ") %>%
    mutate(value = 1) %>% 
    spread(categories, value, fill = 0)